In [1]:

%matplotlib widget
import rospy
import ipywidgets as widgets
import actionlib
import actionlib.msg
import assignment_2_2023.msg
import matplotlib.pyplot as plt
#%matplotlib inline
import numpy as np


from assignment_2_2023.srv import LastTarget, LastTargetRequest
from nav_msgs.msg import Odometry
from ipywidgets import Layout
from matplotlib.animation import FuncAnimation


last_positions_x=[]
last_positions_y=[]

setted_target_x=[]
setted_target_y=[]


canceled_target_x=[]
canceled_target_y=[]


arrived_x=[]
arrived_y=[]

In [2]:
def feedback_clbk(feedback):
    global success
    if feedback.stat=="Target reached!":
        
        arrived_x.append(des_x.value)
        arrived_y.append(des_y.value)
    #print(feedback.stat)

def on_Send_Goal_Button_Clicked(b):
    client.wait_for_server()
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = des_x.value
    goal.target_pose.pose.position.y = des_y.value
    setted_target_x.append(des_x.value)
    setted_target_y.append(des_y.value)
    #print(goal)
    client.send_goal(goal,feedback_cb=feedback_clbk)
    

In [3]:
def on_Cancel_Button_Clicked(b):
    last_positions_x.append(des_x.value)
    last_positions_y.append(des_y.value)
    canceled_target_x.append(des_x.value)
    canceled_target_y.append(des_y.value)
    client.cancel_goal()



In [4]:
'''
des_x=widgets.BoundedFloatText(min=-9, max=9.0,description='x position')
display(des_x)

des_y=widgets.BoundedFloatText(min=-9, max=9.0, description='y position')
display(des_y)
rospy.init_node('action_client')
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)

sendGoal = widgets.Button(description="Send Goal !") 
cancelGoal=widgets.Button(description="Cancel Goal !")
sendGoal.on_click(on_Send_Goal_Button_Clicked)
cancelGoal.on_click(on_Cancel_Button_Clicked)
display(sendGoal)
display(cancelGoal)

'''



'\ndes_x=widgets.BoundedFloatText(min=-9, max=9.0,description=\'x position\')\ndisplay(des_x)\n\ndes_y=widgets.BoundedFloatText(min=-9, max=9.0, description=\'y position\')\ndisplay(des_y)\nrospy.init_node(\'action_client\')\nclient = actionlib.SimpleActionClient(\'/reaching_goal\', assignment_2_2023.msg.PlanningAction)\n\nsendGoal = widgets.Button(description="Send Goal !") \ncancelGoal=widgets.Button(description="Cancel Goal !")\nsendGoal.on_click(on_Send_Goal_Button_Clicked)\ncancelGoal.on_click(on_Cancel_Button_Clicked)\ndisplay(sendGoal)\ndisplay(cancelGoal)\n\n'

In [5]:
def clbk_odom(msg):
# Callback function that process incoming Odometry messages,
# extract informations about: position x, position y, linear velocity x and
# angular velocity z; Save these informations in a custom message Info()
# and pubblish that informations on /robot_pos_vel topic.

# Initialize a new message
# The struct of Info() is (x,y,vel_x,vel_z)
    global pos_x,pos_y,vel_x,vel_z

    pos_x = msg.pose.pose.position.x
    pos_y = msg.pose.pose.position.y
    vel_x = msg.twist.twist.linear.x 		# linear velocity along x axis
    vel_z = msg.twist.twist.angular.z		# angular velocity around z axis
   
    

    
    


In [8]:

class Visualizer:
    def __init__(self):
        self.fig,self.ax=plt.subplots(1,2)
        #self.fig2,self.ax2=plt.subplots()

        self.ln=self.ax[0].plot([],[],'ro')
        self.x_data, self.y_data = [] , []
        self.ax[0].set_aspect('equal')
        self.ax[1].set_aspect('auto')

        self.bar_val= [0,0,0]
        self.bar = self.ax[1].bar(['Send', 'reached' ,'Cancel'], self.bar_val, color=['blue','green','red'])  # Create the bar plot

    
    def plot_init(self):
        self.ax[0].set_xlim(-10,10)
        self.ax[0].set_ylim(-10,10)
       
        return self.ln
    
    #def clback_odom(self,msg):
        #global pos_x,pos_y,vel_x,vel_z
     #   self.x_data.append(msg.pose.pose.position.x)
     #  self.y_data.append(msg.pose.pose.position.y)
    
    #vel_x = msg.twist.twist.linear.x 		# linear velocity along x axis
    #vel_z = msg.twist.twist.angular.z		# angular velocity around z axis
   
    def update_plot(self,frame):
        #plt.plot(self.x_data,self.y_data)
        self.x_data.append(pos_x)
        self.y_data.append(pos_y)
        setted,canceled=0,0
        if(len(setted_target_x)!=0): 
            self.ax[0].plot(setted_target_x[-1], setted_target_y[-1], marker = 'o', color = 'blue' )
        if(len(canceled_target_y)!=0):
            self.ax[0].plot(canceled_target_x[-1], canceled_target_y[-1], marker = 'X', color = 'red' )
        if(len(canceled_target_y)!=0):
            self.ax[0].plot(arrived_x[-1], arrived_y[-1], marker = '+', color = 'green' )
        
        
        self.ax[0].plot(self.x_data,self.y_data)
        self.bar_val=[len(setted_target_x),len(canceled_target_x)]
        
        new_heights=[len(setted_target_x),len(arrived_x),len(canceled_target_y)]
        #self.ax2=plt.bar(x,y)
        
        for i, b in enumerate(self.bar):  # Update the bar plot
            b.set_height(new_heights[i])
            self.ax[1].set_ylim(0,max(self.bar_val) + 1 )
        return self.ln,self.bar

    
            

In [9]:
#rospy.init_node('Visualizer')

des_x=widgets.BoundedFloatText(min=-9, max=9.0,description='x position')
display(des_x)

des_y=widgets.BoundedFloatText(min=-9, max=9.0, description='y position')
display(des_y)
rospy.init_node('action_client')
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)

sendGoal = widgets.Button(description="Send Goal !") 
cancelGoal=widgets.Button(description="Cancel Goal !")
sendGoal.on_click(on_Send_Goal_Button_Clicked)
cancelGoal.on_click(on_Cancel_Button_Clicked)
display(sendGoal)
display(cancelGoal)

vis=Visualizer()
sub = rospy.Subscriber('/odom', Odometry, clbk_odom)
ani=FuncAnimation(vis.fig,vis.update_plot,init_func=vis.plot_init,interval=50)





plt.show(block=True)

BoundedFloatText(value=0.0, description='x position', max=9.0, min=-9.0)

BoundedFloatText(value=0.0, description='y position', max=9.0, min=-9.0)

Button(description='Send Goal !', style=ButtonStyle())

Button(description='Cancel Goal !', style=ButtonStyle())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
success
